## new GGTTIR analsis script 26 september 2019

### import libraries

In [101]:
import pandas as pd
import re
import xlsxwriter

In [53]:
#functions:
#excluding parenthesis
def drop_parenthesis(thinktank):
    parenthesis = re.findall('\(.*?\)',thinktank)
    for p in parenthesis:
        thinktank = thinktank.replace(p,'')
        thinktank = thinktank.strip()
    return thinktank, parenthesis

#count years
def count_years(year_list):
    return len(year_list)


In [91]:
def convert_list(year_list):
    clean_list = []
    for year in year_list:
        year = int(year)
        clean_list.append(year)
    return clean_list

def dropdupl(year_list):
    return list(dict.fromkeys(year_list))
    

In [55]:
def timespan(year_list):
    start = int(year_list[0])
    
    end = int(year_list[-1])
    return start,end

#### import files:

In [56]:
file_path = '../mcgann_ggttir_aktuell.xlsx'

In [57]:
df_regions = pd.read_excel(file_path,sheet_name='regions')
df_special_category = pd.read_excel(file_path,sheet_name='special_category')
df_reserch_area = pd.read_excel(file_path,sheet_name='research_area')

In [59]:
df_regions.head()

,Rank,Think Tank,clean_think_tank_name,Category,Year,Country,thinktanks_clean,parenthesis
0,1,brookings institution (united states),brookings institution (united states),Top 30 Think Tanks United States,2008,United States,brookings institution,[(united states)]
1,2,council on foreign relations,council on foreign relations,Top 30 Think Tanks United States,2008,United States,council on foreign relations,[]
2,3,carnegie endowment for international peace,carnegie endowment for international peace,Top 30 Think Tanks United States,2008,United States,carnegie endowment for international peace,[]
3,4,rand corporation,rand corporation,Top 30 Think Tanks United States,2008,United States,rand corporation,[]
4,5,heritage foundation,heritage foundation,Top 30 Think Tanks United States,2008,United States,heritage foundation,[]


In [58]:
#extract parenthesis from region df:
df_regions['thinktanks_clean'],  df_regions['parenthesis']= zip(*df_regions['clean_think_tank_name'].apply(drop_parenthesis))

#extract parenthesis from region df:
df_special_category['thinktanks_clean'],  df_special_category['parenthesis']= zip(*df_special_category['clean_think_tank_name'].apply(drop_parenthesis))

#extract parenthesis from region df:
df_reserch_area['thinktanks_clean'],  df_reserch_area['parenthesis']= zip(*df_reserch_area['clean_think_tank_name'].apply(drop_parenthesis))

In [80]:
df_sc_cats = df_special_category.groupby('Category')['Year'].apply(list).reset_index()
df_regions_cats = df_regions.groupby('Category')['Year'].apply(list).reset_index()
df_reserch_area_cats = df_reserch_area.groupby('Category')['Year'].apply(list).reset_index()

In [94]:
##Special Categories:

#clean year-list:
df_sc_cats['Year'] = df_sc_cats['Year'].apply(convert_list)

#drop duplicates
df_sc_cats['Year'] = df_sc_cats['Year'].apply(dropdupl)

#get number of years:
df_sc_cats['nr_of_years'] = df_sc_cats['Year'].apply(count_years)

#get the timespan
df_sc_cats['from'],  df_sc_cats['till'] = zip(*df_sc_cats['Year'].apply(timespan))

In [100]:
df_sc_cats.head(1)

,Category,Year,nr_of_years,from,till
0,",Best Think Tanks with Political Party Affilia...",[2012],1,2012,2012


In [95]:
##Regions
#get number of years:
df_regions_cats['nr_of_years'] = df_regions_cats['Year'].apply(count_years)

#drop duplicates
df_regions_cats['Year'] = df_regions_cats['Year'].apply(dropdupl)

#clean year-list:
df_regions_cats['Year'] = df_regions_cats['Year'].apply(convert_list)

#get the timespan
df_regions_cats['from'],  df_regions_cats['till'] = zip(*df_regions_cats['Year'].apply(timespan))

In [ ]:
df_regions_cats.head(1)

In [96]:
## Research Areas:
#get number of years:
df_reserch_area_cats['nr_of_years'] = df_reserch_area_cats['Year'].apply(count_years)

#drop duplicates
df_reserch_area_cats['Year'] = df_reserch_area_cats['Year'].apply(dropdupl)

#clean year-list:
df_reserch_area_cats['Year'] = df_reserch_area_cats['Year'].apply(convert_list)

#get the timespan
df_reserch_area_cats['from'],  df_reserch_area_cats['till'] = zip(*df_reserch_area_cats['Year'].apply(timespan))

In [99]:
df_reserch_area_cats.head(1)

,Category,Year,nr_of_years,from,till
0,List of Food Security Think Tanks (Alphabetica...,[2018],133,2018,2018


## 2. Data Analysis

## 2.1 Categories over time:

In [107]:
df_sc_cats.sort_values(by='nr_of_years',ascending=False).head(2)

,Category,Year,nr_of_years,from,till
6,Best Government Affiliated Think Tanks,"[2010, 2011, 2012, 2013, 2014, 2015, 2016, 201...",9,2010,2018
71,Think Tanks with the Most Innovative Policy Id...,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]",8,2011,2018


In [48]:
print('Special Categories entails {} individual categories which have been used between 2010 and 2018'.format(len(df_sc_categories)))

Special Categories entails 79 individual categories which have been used between 2010 and 2018


In [102]:
writer = pd.ExcelWriter('../results/categories_over_time.xlsx', engine='xlsxwriter')
df_regions_cats.to_excel(writer,sheet_name='regions')
df_reserch_area_cats.to_excel(writer,sheet_name='research_areas')
df_sc_cats.to_excel(writer,sheet_name='special_categories')

writer.save()

In [49]:
df_unique_categories = df_special_category.groupby(['Year'])['Category'].unique()
df_unique_categories = df_unique_categories.to_frame().reset_index()

In [128]:
df_unique_categories['nr_categories'] = df_unique_categories['Category']

,Year,Category
0,2008.0,[Think Tanks with the Most Innovative Policy/I...
1,2009.0,[Think Tanks with the Most Innovative Policy/I...
2,2010.0,[Think Tanks with the Most Innovative Policy/I...
3,2011.0,[Think Tanks with the Most Innovative Policy I...
4,2012.0,[Think Tanks with the Most Innovative Policy I...
5,2013.0,"[Best Advocacy Campaign, Best For Profit Think..."
6,2014.0,"[Best Advocacy Campaign, Best For Profit Think..."
7,2015.0,"[Best Advocacy Campaign, Best For Profit Think..."
8,2016.0,"[Best Advocacy Campaign, Best For Profit Think..."
9,2017.0,"[Best Advocacy Campaign, Best For Profit Think..."
